In [ ]:
!pip install mne

     |████████████████████████████████| 7.4 MB 4.7 MB/s 


In [ ]:
import matplotlib
import pathlib
import glob

import mne
print(mne.__version__)

from scipy.io import loadmat, savemat
import os
import numpy as np
import pandas as pd

0.24.1


In [ ]:
df = np.load('/content/drive/MyDrive/eeg/New problem/meditation_data.npz')

In [ ]:
X = df["X"]
y = df["y"]

In [ ]:
del(df)

In [ ]:
import tensorflow as tf

In [ ]:
y = tf.keras.utils.to_categorical(y, 4)

In [ ]:
data = X

In [ ]:
X = np.zeros((data.shape[0], data.shape[2], data.shape[1]))
for i in range(data.shape[0]):
    X[i] = np.transpose(data[i])

In [ ]:
del(data)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.30,shuffle=True,stratify=y,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_temp,y_temp,test_size=0.50,shuffle=True,stratify=y_temp,random_state=42)

In [ ]:
del(X)
del(y)

In [ ]:
import tensorflow as tf 
from keras import Model
from keras.layers import Input, Dense, Bidirectional
from keras.layers.recurrent import LSTM
import numpy as np

In [ ]:
from tensorflow.keras.layers import Conv1D, LeakyReLU, MaxPool1D, Conv2D, Flatten
from tensorflow.keras.models import Sequential

In [ ]:
inp_shape = (513, 32)
ip = tf.keras.Input(shape = inp_shape)
conv1 = tf.keras.layers.Conv1D(32, 3,)(ip)
rel = tf.keras.layers.LeakyReLU()(conv1)
conv2 = tf.keras.layers.Conv1D(16, 3,)(rel)
rel2 = tf.keras.layers.LeakyReLU()(conv2)
lstm1 = tf.keras.layers.LSTM(16, return_sequences=True)(rel2)
rel10 = tf.keras.layers.LeakyReLU()(lstm1)
                    
lstm2 = tf.keras.layers.LSTM(16, return_sequences=True)(rel10)
rel11 = tf.keras.layers.LeakyReLU()(lstm2)
# lstm = tf.keras.layers.LSTM(16, return_sequences=True)(ip)
flatten = tf.keras.layers.Flatten()(rel11)
dense1 = tf.keras.layers.Dense(units = 256, kernel_initializer = 'random_normal')(flatten)
leakyRelu = tf.keras.layers.LeakyReLU()(dense1)

drop1 = tf.keras.layers.Dropout(.4)(leakyRelu)
dense2 = tf.keras.layers.Dense(units = 32, kernel_initializer = 'random_normal')(drop1)
leakyRelu2 = tf.keras.layers.LeakyReLU()(dense2)
drop2 = tf.keras.layers.Dropout(.4)(dense2)
out = tf.keras.layers.Dense(4, activation = 'softmax')(drop2)

cnn = tf.keras.Model(inputs = ip, outputs = out)
cnn.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 513, 32)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 511, 32)           3104      
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 511, 32)           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 509, 16)           1552      
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 509, 16)           0         
                                                                 
 lstm_2 (LSTM)               (None, 509, 16)           2112      
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 509, 16)           0   

In [ ]:
cnn.compile(loss = 'categorical_crossentropy', metrics = ['acc'], optimizer = 'adam')

In [ ]:
history = cnn.fit(X_train,y_train,epochs=20,batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
542/542 [==============================] - 100s 176ms/step - loss: 1.3882 - acc: 0.2783 - val_loss: 1.3465 - val_acc: 0.2963
Epoch 2/20
542/542 [==============================] - 94s 173ms/step - loss: 1.3367 - acc: 0.3141 - val_loss: 1.3259 - val_acc: 0.3178
Epoch 3/20
542/542 [==============================] - 94s 173ms/step - loss: 1.3049 - acc: 0.3298 - val_loss: 1.2887 - val_acc: 0.3310
Epoch 4/20
542/542 [==============================] - 94s 173ms/step - loss: 1.2760 - acc: 0.3592 - val_loss: 1.2717 - val_acc: 0.3784
Epoch 5/20
542/542 [==============================] - 94s 173ms/step - loss: 1.2575 - acc: 0.3773 - val_loss: 1.2560 - val_acc: 0.3671
Epoch 6/20
542/542 [==============================] - 94s 173ms/step - loss: 1.2425 - acc: 0.3862 - val_loss: 1.2454 - val_acc: 0.3674
Epoch 7/20
542/542 [==============================] - 94s 174ms/step - loss: 1.2105 - acc: 0.4095 - val_loss: 1.2661 - val_acc: 0.3725
Epoch 8/20
542/542 [==============================] - 

In [ ]:
model3 = Sequential()
model3.add(Conv1D(filters = 16, kernel_size = 120, strides = 6, padding = 'same', input_shape=(513,32)))
model3.add(LeakyReLU())
#model2.add(MaxPool1D(pool_size = 2, strides = 1))
model3.add(Conv1D(filters = 16, kernel_size = 3, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(Conv1D(filters = 32, kernel_size = 1, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(Conv1D(filters = 32, kernel_size = 1, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(Conv1D(filters = 64, kernel_size = 2, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(Conv1D(filters = 64, kernel_size = 1, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(Conv1D(filters = 128, kernel_size = 2, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(Conv1D(filters = 128, kernel_size = 2, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(Conv1D(filters = 256, kernel_size = 3, strides = 1, padding = 'same'))
model3.add(LeakyReLU())
model3.add(LSTM(units=16))
model3.add(Flatten())
model3.add(Dense(32, activation = 'softmax'))


model3.add(Dense(4, activation = 'softmax'))

In [ ]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_42 (Conv1D)          (None, 86, 16)            61456     
                                                                 
 leaky_re_lu_52 (LeakyReLU)  (None, 86, 16)            0         
                                                                 
 conv1d_43 (Conv1D)          (None, 86, 16)            784       
                                                                 
 leaky_re_lu_53 (LeakyReLU)  (None, 86, 16)            0         
                                                                 
 conv1d_44 (Conv1D)          (None, 86, 32)            544       
                                                                 
 leaky_re_lu_54 (LeakyReLU)  (None, 86, 32)            0         
                                                                 
 conv1d_45 (Conv1D)          (None, 86, 32)           

In [ ]:
model3.fit(X_train,y_train,epochs=50,batch_size=32, validation_data=(X_val, y_val))